In [1]:
using Flux, Plots, Statistics, MLDatasets, Images, TimeSeries, CSV, DataFrames

In [2]:
df = CSV.read("BAJAJFINSV.csv", DataFrame)
# vwap = df.VWAP

3201-element Vector{Float64}:
   548.85
   572.15
   618.37
   659.6
   636.41
   670.51
   665.87
   602.36
   610.65
   628.44
   624.95
   653.65
   662.71
     ⋮
  9587.77
  9694.01
  9823.88
  9501.23
  9824.26
  9746.74
  9956.69
  9995.72
 10036.76
 10445.96
 10980.4
 11081.78

In [14]:
x=df.VWAP[:,1] # convert to Float32 if possible
N=length(df.VWAP[:,1])
size(x,1)
N

3201

In [4]:
# Do not run
m = Flux.Chain(
            Chain(
                LSTM(N, 10),
                Dense(10, 1)),
            Chain(Flux.Recur(Flux.LSTMCell(N, 10)), Dense(10, 1)))
y = m(x)

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = LSTMCell(3201 => 10)
│   summary(x) = "3201-element Vector{Float64}"
└ @ Flux ~/.julia/packages/Flux/uCLgc/src/layers/stateless.jl:50


LoadError: DimensionMismatch("layer LSTMCell(3201 => 10) expects size(input, 1) == 3201, but got 1-element Vector{Float32}")

In [25]:
n = RNN(2, 5)  # or equivalently RNN(2 => 5)
Flux.Recur(
  Flux.RNNCell(2 => 5, tanh),                # 45 parameters
)

n({1, 2})

LoadError: syntax: { } vector syntax is discontinued around In[25]:6

In [22]:
m = Chain(Flux.RNN(1 => 5), Dense(5 => 1))
Chain(
  Flux.Recur(
    Flux.RNNCell(2 => 5, tanh),              # 45 parameters
  ),
  Dense(5 => 1),                        # 6 parameters
)
#y = m(x)

Chain(
  Recur(
    RNNCell(2 => 5, tanh),              # 45 parameters
  ),
  Dense(5 => 1),                        # 6 parameters
)         # Total: 6 trainable arrays, 51 parameters,
          # plus 1 non-trainable, 5 parameters, summarysize 580 bytes.

In [17]:
function loss(xs, ys)
println(size(xs))
println(size(ys))
l = sum((m(xs)-ys).^2)
return l
end



(3201,)
(1,)


0.0f0

In [23]:
[m(xi) for xi in x]

LoadError: MethodError: no method matching (::Flux.RNNCell{typeof(tanh), Matrix{Float32}, Matrix{Float32}, Vector{Float32}, Matrix{Float32}})(::Matrix{Float32}, ::Float64)
[0mClosest candidates are:
[0m  (::Flux.RNNCell{F, I, H, V, var"#s486"} where var"#s486"<:AbstractMatrix{T})(::Any, [91m::Union{OneHotArrays.OneHotArray, AbstractVecOrMat{var"#s485"} where var"#s485"<:AbstractFloat}[39m) where {F, I, H, V, T} at /Users/irene/.julia/packages/Flux/uCLgc/src/layers/recurrent.jl:203

In [19]:
opt = ADAM(0.01)
ps = Flux.params(m)
data = x
Flux.train!(loss, ps, data, opt)
loss(x, y)

LoadError: MethodError: no method matching loss(::Float64)
[0mClosest candidates are:
[0m  loss(::Any, [91m::Any[39m) at In[17]:1

In [16]:
# evalcb = () → @show loss(df.VWAP, y)

(3201,)
(1,)
loss(df.VWAP, y) = 0.0f0


LoadError: UndefVarError: → not defined